# Homework 3

**Name:** Omar Alejandro Guzmán Munguía

**e-mail:** omar.guzman5063@alumnos.udg.mx

# MODULES

In [2]:
# Load modules
import numpy as np
import math

import plotly.graph_objects as go

from scipy.stats import exponweib
from scipy.spatial import distance

import pandas as pd

# CLASSES

In [4]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# Activity 1: Path length - (BM1 vs BM2 vs CRW)

- Write a function that returns a **Brownian Motion (BM)** trajectory in **pandas** df.
- Write a function that returns a **Correlated Random Walk (CRW)** trajectory in **pandas** df.
- Write a function that returns the **path length** for a given trajectory.
- **Compare** at least the **path lengt** of **three** trajectories as shown in the figure below.
- Display the results using **plotly.**

# FUNCTIONS

In [ ]:
#####################################################################################
# Brownian motion trajectoy
#####################################################################################
def bm_2d(n_steps=1000, speed=5, s_pos=[0,0]):
    """
    Arguments:
        n_steps: Number of steps in the trajectory.
        speed: Speed of the trajectory.
        s_pos: Starting position of the trajectory.
    Returns:
        BM_2d_df:  DataFrame with the x and y positions of the trajectory.
    """
    
    # Init velocity vector
    velocity =Vec2d(speed,0)
    
    # Init DF
    BM_2d_df = pd.DataFrame(columns=['x_pos','y_pos'])    
    # Add initial position
    temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])    
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
    
    # Generate the trajectory
    for i in range(n_steps-1):        
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)
    
        temp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x, 'y_pos':BM_2d_df.y_pos[i]+velocity.y}])    
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
        
    return BM_2d_df

In [ ]:
#####################################################################################
# Correlated Random Walk (CRW)
#####################################################################################

def crw_trajectory(n_steps=100, speed=5, correlation_coeff=0.9, s_pos=0.0):
    """
    Arguments:
        n_steps: Number of steps in the trajectory.
        speed: Length of each step.
        correlation_coeff: Correlation coefficient (between -1 and 1) that controls the correlation between consecutive steps.
        s_pos: Initial direction of movement in radians (default is 0.0).
    Returns:
        crw_df: A pandas DataFrame with columns ['x', 'y'] representing the trajectory.
    """
    # Initialize the trajectory with the starting point at the origin
    trajectory = [Vec2d(0.0, 0.0)]
    
    # Initialize the current direction
    current_direction = s_pos
    
    for _ in range(n_steps):
        # Generate a random angle change based on the correlation coefficient
        angle_change = np.random.normal(0, np.sqrt(1 - correlation_coeff**2))
        
        # Update the current direction
        current_direction += angle_change
        
        # Calculate the next step vector
        next_step = Vec2d(speed, 0).rotated(current_direction)
        
        # Update the trajectory
        next_position = trajectory[-1] + next_step
        trajectory.append(next_position)
    
    # Convert the trajectory to a pandas DataFrame
    crw_df = pd.DataFrame([(pos.x, pos.y) for pos in trajectory], columns=['x', 'y'])
    
    return crw_df